In [ ]:
# Local live dat adquisition (into csv)

import os
import json
import pandas as pd
import numpy as np
import redpitaya_scpi as scpi
import time
import datetime
import re
import sys
from scipy import stats
from scipy.signal import welch
import logging

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = ["ip_address", "decimation_factor", "base_dir", "session_delay_seconds"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

def setup_logging(log_path):
    """
    Configure logging settings.
    """
    logging.basicConfig(
        filename=log_path,
        filemode='a',
        format='%(asctime)s - %(levelname)s - %(message)s',
        level=logging.INFO
    )

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(signal, decimation_factor):
    """
    Compute statistical features from the raw signal data.
    """
    try:
        # Remove DC component
        signal = signal - np.mean(signal)
        
        # Apply Welch's method to estimate power spectral density
        sampling_rate = 125e6 / decimation_factor  # Adjust based on decimation factor
        frequencies, power_spectrum = welch(signal, fs=sampling_rate, nperseg=1024)
        
        # Convert power spectrum to dBm assuming 50 ohms impedance
        Z_ref = 50  # Reference impedance in ohms
        power_dbm = 10 * np.log10(power_spectrum / Z_ref)
        
        # Filter frequencies (10kHz to 100kHz)
        mask = (frequencies >= 10e3) & (frequencies <= 100e3)
        filtered_power_dbm = power_dbm[mask]
        
        # Calculate statistical features
        features = {}
        features['entropy'] = stats.entropy(np.abs(filtered_power_dbm))
        features['skewness'] = stats.skew(filtered_power_dbm)
        features['interquartile_range'] = np.percentile(filtered_power_dbm, 75) - np.percentile(filtered_power_dbm, 25)
        features['kurtosis'] = stats.kurtosis(filtered_power_dbm)
        features['percentile_75'] = np.percentile(filtered_power_dbm, 75)
        features['range'] = np.ptp(filtered_power_dbm)
        features['maximum'] = np.max(filtered_power_dbm)
        features['median'] = np.median(filtered_power_dbm)
        features['percentile_90'] = np.percentile(filtered_power_dbm, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(filtered_power_dbm - np.mean(filtered_power_dbm)))
        
        return features
    except Exception as e:
        logging.error(f"Error computing statistical features: {e}")
        # Return NaN for all features in case of error
        features = {key: np.nan for key in ['entropy', 'skewness', 'interquartile_range', 'kurtosis',
                                           'percentile_75', 'range', 'maximum', 'median',
                                           'percentile_90', 'mean_absolute_deviation']}
        return features

# ---------------------------
# Data Acquisition Functions
# ---------------------------

def initialize_live_data_csv(live_data_path, feature_columns):
    """
    Initialize the live_data.csv file with headers if it doesn't exist.
    """
    if not os.path.exists(live_data_path):
        with open(live_data_path, 'w') as f:
            # Define the header based on feature_columns and timestamp
            header = ['date', 'timestamp'] + feature_columns
            f.write(','.join(header) + '\n')
        logging.info(f"Initialized live data CSV at {live_data_path}")
    else:
        logging.info(f"Live data CSV already exists at {live_data_path}")

def append_features_to_live_data(live_data_path, timestamp, features):
    """
    Append a new row of features to live_data.csv.
    """
    try:
        # Extract date from timestamp
        date_str = timestamp.split('T')[0]
        
        # Prepare feature values in the specified order
        feature_values = [str(features[col]) for col in features]
        
        # Create the row with date, timestamp, and feature values
        row = [date_str, timestamp] + feature_values
        with open(live_data_path, 'a') as f:
            f.write(','.join(row) + '\n')
        logging.info(f"Appended features at {timestamp} to live data CSV")
    except Exception as e:
        logging.error(f"Error appending features to live data CSV: {e}")

def data_acquisition(config):
    """
    Main data acquisition function.
    """
    # Extract configurations
    IP = config['ip_address']
    decimation_factor = config['decimation_factor']
    base_dir = config['base_dir']
    session_delay = config['session_delay_seconds']
    
    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(IP)
        logging.info(f"Connected to Red Pitaya at {IP}")
        print(f"Connected to Red Pitaya at {IP}")
    except Exception as e:
        logging.error(f"Failed to connect to Red Pitaya at {IP}: {e}")
        print(f"Failed to connect to Red Pitaya at {IP}: {e}")
        sys.exit(1)
    
    # Define live_data.csv path
    live_data_path = os.path.join(base_dir, 'live_data.csv')
    
    # Define feature columns
    feature_columns = [
        'ch1_entropy', 'ch1_skewness', 'ch1_interquartile_range', 'ch1_kurtosis',
        'ch1_percentile_75', 'ch1_range', 'ch1_maximum', 'ch1_median',
        'ch1_percentile_90', 'ch1_mean_absolute_deviation',
        'ch2_entropy', 'ch2_skewness', 'ch2_interquartile_range', 'ch2_kurtosis',
        'ch2_percentile_75', 'ch2_range', 'ch2_maximum', 'ch2_median',
        'ch2_percentile_90', 'ch2_mean_absolute_deviation'
    ]
    
    # Initialize live_data.csv
    initialize_live_data_csv(live_data_path, feature_columns)
    
    try:
        while True:
            logging.info("Starting data acquisition cycle.")
            print("Starting data acquisition cycle.")
    
            # Configure Red Pitaya for acquisition
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:TRIG:LEV -1')  # Set trigger level
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                logging.info("Red Pitaya configured successfully.")
                print("Red Pitaya configured successfully.")
            except Exception as e:
                logging.error(f"Error configuring Red Pitaya: {e}")
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration
    
            # Trigger acquisition
            try:
                rp_s.tx_txt('ACQ:TRIG CH1_PE')
                rp_s.tx_txt('ACQ:TRIG CH2_PE')
                time.sleep(1)  # Wait for trigger and acquisition
                logging.info("Acquisition triggered successfully.")
                print("Acquisition triggered successfully.")
            except Exception as e:
                logging.error(f"Error triggering acquisition: {e}")
                print(f"Error triggering acquisition: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration
    
            # Capture data from both channels
            channel_data = {}
            data_acquired = False  # Flag to check if any data was acquired
            for channel in ['CH1', 'CH2']:
                try:
                    rp_s.tx_txt(f'ACQ:SOUR{1 if channel == "CH1" else 2}:DATA?')
                    raw_data = rp_s.rx_txt().strip('{}\n\r').split(',')
                    signal = np.array([float(x) for x in raw_data])
    
                    if signal.size > 0:
                        channel_data[channel] = signal
                        data_acquired = True
                        logging.info(f"Data acquired from {channel}")
                        print(f"Data acquired from {channel}")
                    else:
                        logging.warning(f"No data captured for {channel}")
                        print(f"No data captured for {channel}")
                        channel_data[channel] = np.array([])
                except Exception as e:
                    logging.error(f"Error acquiring data from {channel}: {e}")
                    print(f"Error acquiring data from {channel}: {e}")
                    channel_data[channel] = np.array([])
    
            if data_acquired:
                # Compute features for CH1 and CH2
                features = {}
                for channel in ['CH1', 'CH2']:
                    if channel_data[channel].size > 0:
                        channel_features = compute_statistical_features(channel_data[channel], decimation_factor)
                        # Prefix feature names with channel identifier
                        for key, value in channel_features.items():
                            features[f'{channel.lower()}_{key}'] = value
                    else:
                        # Assign NaN to all features if no data
                        for key in ['entropy', 'skewness', 'interquartile_range', 'kurtosis',
                                   'percentile_75', 'range', 'maximum', 'median',
                                   'percentile_90', 'mean_absolute_deviation']:
                            features[f'{channel.lower()}_{key}'] = np.nan
    
                # Get current timestamp
                timestamp = datetime.datetime.now().isoformat()
    
                # Append features to live_data.csv
                append_features_to_live_data(live_data_path, timestamp, features)
    
                logging.info(f"Features appended to {live_data_path} at {timestamp}")
                print(f"Features appended to {live_data_path} at {timestamp}")
            else:
                logging.warning("No data acquired in this cycle. Skipping feature computation.")
                print("No data acquired in this cycle. Skipping feature computation.")
    
            # Delay before next acquisition cycle
            time.sleep(session_delay)
    except KeyboardInterrupt:
        logging.info("Data acquisition stopped by user.")
        print("\nData acquisition stopped by user.")
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        print(f"An unexpected error occurred: {e}")
    finally:
        logging.info("Data acquisition terminated.")
        print("Data acquisition terminated.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    try:
        config = load_config('config2.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)
    
    log_path = os.path.join(config['base_dir'], 'data_acquisition.log')
    setup_logging(log_path)
    data_acquisition(config)


Connected to Red Pitaya at 192.168.8.162
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
Features appended to live_data.csv at 2024-11-11T10:42:47.572539
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
Features appended to live_data.csv at 2024-11-11T10:42:49.972750
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
Features appended to live_data.csv at 2024-11-11T10:42:52.372810
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
Features appended to live_data.csv at 2024-11-11T10:42:54.789377
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered succ

In [ ]:
# live_data_acquisition.py to AWS

import os
import json
import pandas as pd
import numpy as np
import redpitaya_scpi as scpi
import time
import datetime
import re
import sys
from scipy import stats
from scipy.signal import welch
import requests

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = [
        "ip_address", "decimation_factor", "base_dir",
        "session_delay_seconds", "aws_api_endpoint", "aws_api_key"
    ]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(signal, decimation_factor):
    """
    Compute statistical features from the raw signal data.
    """
    try:
        # Remove DC component
        signal = signal - np.mean(signal)
        
        # Apply Welch's method to estimate power spectral density
        sampling_rate = 125e6 / decimation_factor  # Adjust based on decimation factor
        frequencies, power_spectrum = welch(signal, fs=sampling_rate, nperseg=1024)
        
        # Convert power spectrum to dBm assuming 50 ohms impedance
        Z_ref = 50  # Reference impedance in ohms
        power_dbm = 10 * np.log10(power_spectrum / Z_ref)
        
        # Filter frequencies (10kHz to 100kHz)
        mask = (frequencies >= 10e3) & (frequencies <= 100e3)
        filtered_power_dbm = power_dbm[mask]
        
        # Calculate statistical features
        features = {}
        features['entropy'] = stats.entropy(np.abs(filtered_power_dbm))
        features['skewness'] = stats.skew(filtered_power_dbm)
        features['interquartile_range'] = np.percentile(filtered_power_dbm, 75) - np.percentile(filtered_power_dbm, 25)
        features['kurtosis'] = stats.kurtosis(filtered_power_dbm)
        features['percentile_75'] = np.percentile(filtered_power_dbm, 75)
        features['range'] = np.ptp(filtered_power_dbm)
        features['maximum'] = np.max(filtered_power_dbm)
        features['median'] = np.median(filtered_power_dbm)
        features['percentile_90'] = np.percentile(filtered_power_dbm, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(filtered_power_dbm - np.mean(filtered_power_dbm)))
        
        return features
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        # Return NaN for all features in case of error
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation'
        ]}
        return features

# ---------------------------
# Data Acquisition Functions
# ---------------------------

def send_features_to_aws(api_endpoint, api_key, timestamp, features):
    """
    Send the features to AWS API Gateway endpoint.
    """
    try:
        # Prepare the payload wrapped in 'body' key
        payload = {
            "body": json.dumps({
                'timestamp': timestamp,
                'features': features
            })
        }
        headers = {
            'Content-Type': 'application/json',
            'x-api-key': api_key  # Include API Key if required
        }
        # Send POST request with the payload
        response = requests.post(api_endpoint, data=json.dumps(payload), headers=headers)
        if response.status_code == 200:
            print(f"{response.text}")
        else:
            print(f"Failed to send features to AWS. Status Code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"Error sending features to AWS: {e}")

def data_acquisition(config):
    """
    Main data acquisition function.
    """
    # Extract configurations
    IP = config['ip_address']
    decimation_factor = config['decimation_factor']
    base_dir = config['base_dir']
    session_delay = config['session_delay_seconds']
    api_endpoint = config['aws_api_endpoint']
    api_key = config['aws_api_key']  # API Key for AWS API Gateway

    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(IP)
        print(f"Connected to Red Pitaya at {IP}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {IP}: {e}")
        sys.exit(1)

    # Define feature columns
    feature_columns = [
        'ch1_entropy', 'ch1_skewness', 'ch1_interquartile_range', 'ch1_kurtosis',
        'ch1_percentile_75', 'ch1_range', 'ch1_maximum', 'ch1_median',
        'ch1_percentile_90', 'ch1_mean_absolute_deviation',
        'ch2_entropy', 'ch2_skewness', 'ch2_interquartile_range', 'ch2_kurtosis',
        'ch2_percentile_75', 'ch2_range', 'ch2_maximum', 'ch2_median',
        'ch2_percentile_90', 'ch2_mean_absolute_deviation'
    ]

    try:
        while True:
            print("Starting data acquisition cycle.")

            # Configure Red Pitaya for acquisition
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:TRIG:LEV -1')  # Set trigger level
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                print("Red Pitaya configured successfully.")
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Trigger acquisition
            try:
                rp_s.tx_txt('ACQ:TRIG CH1_PE')
                rp_s.tx_txt('ACQ:TRIG CH2_PE')
                time.sleep(1)  # Wait for trigger and acquisition
                print("Acquisition triggered successfully.")
            except Exception as e:
                print(f"Error triggering acquisition: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Capture data from both channels
            channel_data = {}
            data_acquired = False  # Flag to check if any data was acquired
            for channel in ['CH1', 'CH2']:
                try:
                    rp_s.tx_txt(f'ACQ:SOUR{1 if channel == "CH1" else 2}:DATA?')
                    raw_data = rp_s.rx_txt().strip('{}\n\r').split(',')
                    signal = np.array([float(x) for x in raw_data])

                    if signal.size > 0:
                        channel_data[channel] = signal
                        data_acquired = True
                        print(f"Data acquired from {channel}")
                    else:
                        print(f"No data captured for {channel}")
                        channel_data[channel] = np.array([])
                except Exception as e:
                    print(f"Error acquiring data from {channel}: {e}")
                    channel_data[channel] = np.array([])

            if data_acquired:
                # Compute features for CH1 and CH2
                features = {}
                for channel in ['CH1', 'CH2']:
                    if channel_data[channel].size > 0:
                        channel_features = compute_statistical_features(channel_data[channel], decimation_factor)
                        # Prefix feature names with channel identifier
                        for key, value in channel_features.items():
                            features[f'{channel.lower()}_{key}'] = value
                    else:
                        # Assign NaN to all features if no data
                        for key in [
                            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
                            'percentile_75', 'range', 'maximum', 'median',
                            'percentile_90', 'mean_absolute_deviation'
                        ]:
                            features[f'{channel.lower()}_{key}'] = np.nan

                # Get current timestamp
                timestamp = datetime.datetime.now().isoformat()

                # Send features to AWS
                send_features_to_aws(api_endpoint, api_key, timestamp, features)

            # Delay before next acquisition cycle
            time.sleep(session_delay)
    except KeyboardInterrupt:
        print("\nData acquisition stopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Data acquisition terminated.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)

    data_acquisition(config)


Connected to Red Pitaya at 192.168.8.214
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
{"statusCode": 200, "body": "{\"message\": \"Data saved to DynamoDB\", \"timestamp\": \"2024-11-16T11:48:05.436552\"}"}
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
{"statusCode": 200, "body": "{\"message\": \"Data saved to DynamoDB\", \"timestamp\": \"2024-11-16T11:48:12.612520\"}"}

Data acquisition stopped by user.
Data acquisition terminated.


In [ ]:
import os
import csv
import json
import pandas as pd
import numpy as np
import redpitaya_scpi as scpi
import time
import datetime
import sys
from scipy import stats
from scipy.signal import welch
import requests
import joblib
from tensorflow.keras.models import load_model

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = [
        "ip_address", "decimation_factor", "base_dir",
        "session_delay_seconds", "aws_api_endpoint", "aws_api_key",
        "model_path", "scaler_path", "mlb_path"
    ]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(signal, decimation_factor):
    """
    Compute statistical features from the raw signal data.
    """
    try:
        # Remove DC component
        signal = signal - np.mean(signal)
        
        # Apply Welch's method to estimate power spectral density
        sampling_rate = 125e6 / decimation_factor  # Adjust based on decimation factor
        frequencies, power_spectrum = welch(signal, fs=sampling_rate, nperseg=1024)
        
        # Convert power spectrum to dBm assuming 50 ohms impedance
        Z_ref = 50  # Reference impedance in ohms
        power_dbm = 10 * np.log10(power_spectrum / Z_ref)
        
        # Filter frequencies (10kHz to 100kHz)
        mask = (frequencies >= 10e3) & (frequencies <= 100e3)
        filtered_power_dbm = power_dbm[mask]
        
        # Calculate statistical features
        features = {}
        features['entropy'] = stats.entropy(np.abs(filtered_power_dbm))
        features['skewness'] = stats.skew(filtered_power_dbm)
        features['interquartile_range'] = np.percentile(filtered_power_dbm, 75) - np.percentile(filtered_power_dbm, 25)
        features['kurtosis'] = stats.kurtosis(filtered_power_dbm)
        features['percentile_75'] = np.percentile(filtered_power_dbm, 75)
        features['range'] = np.ptp(filtered_power_dbm)
        features['maximum'] = np.max(filtered_power_dbm)
        features['median'] = np.median(filtered_power_dbm)
        features['percentile_90'] = np.percentile(filtered_power_dbm, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(filtered_power_dbm - np.mean(filtered_power_dbm)))
        
        return features
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        # Return NaN for all features in case of error
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation'
        ]}
        return features

# ---------------------------
# Prediction Function
# ---------------------------

def predict_labels(features, model, scaler, mlb):
    """
    Takes a dictionary of features, uses the trained model to predict labels and their probabilities.
    
    Parameters:
    features (dict): A dictionary containing the feature names and their corresponding values.
                     The keys should match the feature columns used during training.
    model: Loaded Keras model.
    scaler: Loaded scaler object.
    mlb: Loaded MultiLabelBinarizer object.
    
    Returns:
    tuple: A tuple containing:
        - predicted_labels (list): A list of predicted labels.
        - label_probabilities (dict): A dictionary mapping label names to their predicted probabilities.
    """
    
    feature_columns = ['ch1_entropy', 'ch1_skewness', 'ch1_interquartile_range',
                       'ch1_kurtosis', 'ch1_percentile_75', 'ch1_range', 'ch1_maximum',
                       'ch1_median', 'ch1_percentile_90', 'ch1_mean_absolute_deviation']
    

    # Ensure all required features are provided
    missing_features = set(feature_columns) - set(features.keys())
    if missing_features:
        raise ValueError(f"The following features are missing: {missing_features}")

    # Create a DataFrame with the feature columns
    X = pd.DataFrame([features], columns=feature_columns)

    # Standardize the features
    X_scaled = scaler.transform(X)

    # Make predictions
    probabilities = model.predict(X_scaled)[0]

    # Apply threshold to get binary predictions
    threshold = 0.5
    predictions_binary = (probabilities >= threshold).astype(int)

    # Ensure predictions_binary is 2D
    predictions_binary = predictions_binary.reshape(1, -1)

    # Get the predicted labels
    predicted_labels = mlb.inverse_transform(predictions_binary)[0]

    # Map label names to probabilities
    label_probabilities = dict(zip(mlb.classes_, probabilities.astype(float)))

    return predicted_labels, label_probabilities

# ---------------------------
# Data Acquisition Functions
# ---------------------------

def send_features_to_aws(api_endpoint, api_key, timestamp, features, label_probabilities):
    """
    Send the features and label probabilities to AWS API Gateway endpoint.
    """
    try:
        # Prepare the payload wrapped in 'body' key
        payload = {
            "body": json.dumps({
                'timestamp': timestamp,
                'features': features,
                'label_probabilities': label_probabilities
            })
        }
        headers = {
            'Content-Type': 'application/json',
            'x-api-key': api_key  # Include API Key if required
        }
        print(payload)
        # Send POST request with the payload
        response = requests.post(api_endpoint, data=json.dumps(payload), headers=headers)
        if response.status_code == 200:
            print(f"{response.text}")
        else:
            print(f"Failed to send data to AWS. Status Code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"Error sending data to AWS: {e}")

def append_payload_to_csv(file_path, timestamp, features, label_probabilities):
    """
    Append the payload to a CSV file.
    """
    # Define the header for the CSV
    header = ['timestamp'] + list(label_probabilities.keys())
    
    # Prepare the row data
    row = [timestamp] + [f"{value:.3f}" if isinstance(value, (int, float)) else value for value in label_probabilities.values()]

    # Check if the file exists
    file_exists = os.path.isfile(file_path)

    try:
        # Open the file in append mode
        with open(file_path, mode='a', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Write the header only if the file does not exist
            if not file_exists:
                writer.writerow(header)

            # Write the row data
            writer.writerow(row)
        
        print(f"Payload appended to {file_path} successfully.")
    except Exception as e:
        print(f"Error writing payload to CSV: {e}")

def data_acquisition(config):
    """
    Main data acquisition function.
    """
    # Extract configurations
    IP = config['ip_address']
    decimation_factor = config['decimation_factor']
    base_dir = config['base_dir']
    session_delay = config['session_delay_seconds']
    api_endpoint = config['aws_api_endpoint']
    api_key = config['aws_api_key']  # API Key for AWS API Gateway
    model_path = config['model_path']
    scaler_path = config['scaler_path']
    mlb_path = config['mlb_path']

    # Load the model, scaler, and mlb once
    if not os.path.exists(model_path):
        print(f"Model file not found at {model_path}")
        sys.exit(1)
    if not os.path.exists(scaler_path):
        print(f"Scaler file not found at {scaler_path}")
        sys.exit(1)
    if not os.path.exists(mlb_path):
        print(f"MLB file not found at {mlb_path}")
        sys.exit(1)

    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    mlb = joblib.load(mlb_path)

    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(IP)
        print(f"Connected to Red Pitaya at {IP}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {IP}: {e}")
        sys.exit(1)

    try:
        while True:
            print("Starting data acquisition cycle.")

            # Configure Red Pitaya for acquisition
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:TRIG:LEV -1')  # Set trigger level
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                print("Red Pitaya configured successfully.")
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Trigger acquisition
            try:
                rp_s.tx_txt('ACQ:TRIG CH1_PE')
                rp_s.tx_txt('ACQ:TRIG CH2_PE')
                time.sleep(1)  # Wait for trigger and acquisition
                print("Acquisition triggered successfully.")
            except Exception as e:
                print(f"Error triggering acquisition: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Capture data from both channels
            channel_data = {}
            data_acquired = False  # Flag to check if any data was acquired
            for channel in ['CH1', 'CH2']:
                try:
                    rp_s.tx_txt(f'ACQ:SOUR{1 if channel == "CH1" else 2}:DATA?')
                    raw_data = rp_s.rx_txt().strip('{}\n\r').split(',')
                    signal = np.array([float(x) for x in raw_data])

                    if signal.size > 0:
                        channel_data[channel] = signal
                        data_acquired = True
                        print(f"Data acquired from {channel}")
                    else:
                        print(f"No data captured for {channel}")
                        channel_data[channel] = np.array([])
                except Exception as e:
                    print(f"Error acquiring data from {channel}: {e}")
                    channel_data[channel] = np.array([])

            if data_acquired:
                # Compute features for CH1 and CH2
                features = {}
                for channel in ['CH1', 'CH2']:
                    if channel_data[channel].size > 0:
                        channel_features = compute_statistical_features(channel_data[channel], decimation_factor)
                        # Prefix feature names with channel identifier
                        for key, value in channel_features.items():
                            features[f'{channel.lower()}_{key}'] = value
                    else:
                        # Assign NaN to all features if no data
                        for key in [
                            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
                            'percentile_75', 'range', 'maximum', 'median',
                            'percentile_90', 'mean_absolute_deviation'
                        ]:
                            features[f'{channel.lower()}_{key}'] = np.nan

                # Get current timestamp
                timestamp = datetime.datetime.now().isoformat()

                # Prepare input data for prediction
                input_data = {
                    'timestamp': timestamp,
                    'features': features
                }

                # Perform prediction
                try:
                    predicted_labels, label_probabilities = predict_labels(features, model, scaler, mlb)
                except ValueError as ve:
                    print(f"Prediction Error: {ve}")
                    label_probabilities = {}
                    predicted_labels = []
                except Exception as e:
                    print(f"An error occurred during prediction: {e}")
                    label_probabilities = {}
                    predicted_labels = []

                append_payload_to_csv('hello.csv', timestamp, features, label_probabilities)

                # Send features and label probabilities to AWS
                send_features_to_aws(api_endpoint, api_key, timestamp, features, label_probabilities)

            # Delay before next acquisition cycle
            time.sleep(session_delay)
    except KeyboardInterrupt:
        print("\nData acquisition stopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Data acquisition terminated.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)

    data_acquisition(config)


Connected to Red Pitaya at 192.168.8.214
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Payload appended to hello.csv successfully.
{'body': '{"timestamp": "2024-11-20T00:41:13.792998", "features": {"ch1_entropy": 6.055716788361036, "ch1_skewness": 1.616728447584021, "ch1_interquartile_range": 12.096909918610152, "ch1_kurtosis": 3.3105306381468242, "ch1_percentile_75": -93.11389653059999, "ch1_range": 57.465780689162514, "ch1_maximum": -54.69610043855744, "ch1_median": -101.3047828301273, "ch1_percentile_90": -86.04776020844729, "ch1_mean_absolute_deviation": 7.6813926389926275, "ch2_entropy": 6.059168244270834, "ch2_skewness": 1.9790593849895894, "ch2_interquartile_range": 8.196300848651305, "ch2_kurtosis": 5.803203638041232, "ch2_percentile_75": -124.11960741807246, "ch2_range": 58.19247853385269, "ch2_maximum": -82.08637690744212, "ch2_median"

# moving avg

In [1]:
import os
import csv
import json
import pandas as pd
import numpy as np
import redpitaya_scpi as scpi
import time
import datetime
import sys
from scipy import stats
from scipy.signal import welch
import requests
import joblib
from tensorflow.keras.models import load_model

# ---------------------------
# Configuration and Setup
# ---------------------------

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = [
        "ip_address", "decimation_factor", "base_dir",
        "session_delay_seconds", "aws_api_endpoint", "aws_api_key",
        "model_path", "scaler_path", "mlb_path"
    ]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

# ---------------------------
# Statistical Feature Computation
# ---------------------------

def compute_statistical_features(signal, decimation_factor):
    """
    Compute statistical features from the raw signal data.
    """
    try:
        # Remove DC component
        signal = signal - np.mean(signal)
        
        # Apply Welch's method to estimate power spectral density
        sampling_rate = 125e6 / decimation_factor  # Adjust based on decimation factor
        frequencies, power_spectrum = welch(signal, fs=sampling_rate, nperseg=1024)
        
        # Convert power spectrum to dBm assuming 50 ohms impedance
        Z_ref = 50  # Reference impedance in ohms
        power_dbm = 10 * np.log10(power_spectrum / Z_ref)
        
        # Filter frequencies (10kHz to 100kHz)
        mask = (frequencies >= 10e3) & (frequencies <= 100e3)
        filtered_power_dbm = power_dbm[mask]
        
        # Calculate statistical features
        features = {}
        features['entropy'] = stats.entropy(np.abs(filtered_power_dbm))
        features['skewness'] = stats.skew(filtered_power_dbm)
        features['interquartile_range'] = np.percentile(filtered_power_dbm, 75) - np.percentile(filtered_power_dbm, 25)
        features['kurtosis'] = stats.kurtosis(filtered_power_dbm)
        features['percentile_75'] = np.percentile(filtered_power_dbm, 75)
        features['range'] = np.ptp(filtered_power_dbm)
        features['maximum'] = np.max(filtered_power_dbm)
        features['median'] = np.median(filtered_power_dbm)
        features['percentile_90'] = np.percentile(filtered_power_dbm, 90)
        features['mean_absolute_deviation'] = np.mean(np.abs(filtered_power_dbm - np.mean(filtered_power_dbm)))
        
        return features
    except Exception as e:
        print(f"Error computing statistical features: {e}")
        # Return NaN for all features in case of error
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
            'percentile_75', 'range', 'maximum', 'median',
            'percentile_90', 'mean_absolute_deviation'
        ]}
        return features

# ---------------------------
# Prediction Function
# ---------------------------

def predict_labels(features, model, scaler, mlb):
    """
    Takes a dictionary of features, uses the trained model to predict labels and their probabilities.
    
    Parameters:
    features (dict): A dictionary containing the feature names and their corresponding values.
                     The keys should match the feature columns used during training.
    model: Loaded Keras model.
    scaler: Loaded scaler object.
    mlb: Loaded MultiLabelBinarizer object.
    
    Returns:
    tuple: A tuple containing:
        - predicted_labels (list): A list of predicted labels.
        - label_probabilities (dict): A dictionary mapping label names to their predicted probabilities.
    """
    
    feature_columns = ['ch1_entropy', 'ch1_skewness', 'ch1_interquartile_range',
                       'ch1_kurtosis', 'ch1_percentile_75', 'ch1_range', 'ch1_maximum',
                       'ch1_median', 'ch1_percentile_90', 'ch1_mean_absolute_deviation']
    

    # Ensure all required features are provided
    missing_features = set(feature_columns) - set(features.keys())
    if missing_features:
        raise ValueError(f"The following features are missing: {missing_features}")

    # Create a DataFrame with the feature columns
    X = pd.DataFrame([features], columns=feature_columns)

    # Standardize the features
    X_scaled = scaler.transform(X)

    # Make predictions
    probabilities = model.predict(X_scaled)[0]

    # Apply threshold to get binary predictions
    threshold = 0.5
    predictions_binary = (probabilities >= threshold).astype(int)

    # Ensure predictions_binary is 2D
    predictions_binary = predictions_binary.reshape(1, -1)

    # Get the predicted labels
    predicted_labels = mlb.inverse_transform(predictions_binary)[0]

    # Map label names to probabilities
    label_probabilities = dict(zip(mlb.classes_, probabilities.astype(float)))

    return predicted_labels, label_probabilities

# ---------------------------
# Data Acquisition Functions
# ---------------------------

def send_features_to_aws(api_endpoint, api_key, timestamp, features, label_probabilities):
    """
    Send the features and label probabilities to AWS API Gateway endpoint.
    """
    try:
        # Prepare the payload wrapped in 'body' key
        payload = {
            "body": json.dumps({
                'timestamp': timestamp,
                'features': features,
                'label_probabilities': label_probabilities
            })
        }
        headers = {
            'Content-Type': 'application/json',
            'x-api-key': api_key  # Include API Key if required
        }
        print(payload)
        # Send POST request with the payload
        response = requests.post(api_endpoint, data=json.dumps(payload), headers=headers)
        if response.status_code == 200:
            print(f"{response.text}")
        else:
            print(f"Failed to send data to AWS. Status Code: {response.status_code}, Response: {response.text}")
    except Exception as e:
        print(f"Error sending data to AWS: {e}")

def append_payload_to_csv(file_path, timestamp, features, label_probabilities):
    """
    Append the payload to a CSV file.
    """
    # Define the header for the CSV
    header = ['timestamp'] + list(label_probabilities.keys())
    
    # Prepare the row data
    row = [timestamp] + [f"{value:.3f}" if isinstance(value, (int, float)) else value for value in label_probabilities.values()]

    # Check if the file exists
    file_exists = os.path.isfile(file_path)

    try:
        # Open the file in append mode
        with open(file_path, mode='a', newline='') as csvfile:
            writer = csv.writer(csvfile)

            # Write the header only if the file does not exist
            if not file_exists:
                writer.writerow(header)

            # Write the row data
            writer.writerow(row)
        
        print(f"Payload appended to {file_path} successfully.")
    except Exception as e:
        print(f"Error writing payload to CSV: {e}")

def data_acquisition(config):
    """
    Main data acquisition function.
    """
    # Extract configurations
    IP = config['ip_address']
    decimation_factor = config['decimation_factor']
    base_dir = config['base_dir']
    session_delay = config['session_delay_seconds']
    api_endpoint = config['aws_api_endpoint']
    api_key = config['aws_api_key']  # API Key for AWS API Gateway
    model_path = config['model_path']
    scaler_path = config['scaler_path']
    mlb_path = config['mlb_path']

    # Load the model, scaler, and mlb once
    if not os.path.exists(model_path):
        print(f"Model file not found at {model_path}")
        sys.exit(1)
    if not os.path.exists(scaler_path):
        print(f"Scaler file not found at {scaler_path}")
        sys.exit(1)
    if not os.path.exists(mlb_path):
        print(f"MLB file not found at {mlb_path}")
        sys.exit(1)

    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    mlb = joblib.load(mlb_path)

    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(IP)
        print(f"Connected to Red Pitaya at {IP}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {IP}: {e}")
        sys.exit(1)

    previous_features = None  # Initialize previous_features

    try:
        while True:
            print("Starting data acquisition cycle.")

            # Configure Red Pitaya for acquisition
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:TRIG:LEV -1')  # Set trigger level
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                print("Red Pitaya configured successfully.")
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Trigger acquisition
            try:
                rp_s.tx_txt('ACQ:TRIG CH1_PE')
                rp_s.tx_txt('ACQ:TRIG CH2_PE')
                time.sleep(1)  # Wait for trigger and acquisition
                print("Acquisition triggered successfully.")
            except Exception as e:
                print(f"Error triggering acquisition: {e}")
                time.sleep(session_delay)
                continue  # Skip to next iteration

            # Capture data from both channels
            channel_data = {}
            data_acquired = False  # Flag to check if any data was acquired
            for channel in ['CH1', 'CH2']:
                try:
                    rp_s.tx_txt(f'ACQ:SOUR{1 if channel == "CH1" else 2}:DATA?')
                    raw_data = rp_s.rx_txt().strip('{}\n\r').split(',')
                    signal = np.array([float(x) for x in raw_data])

                    if signal.size > 0:
                        channel_data[channel] = signal
                        data_acquired = True
                        print(f"Data acquired from {channel}")
                    else:
                        print(f"No data captured for {channel}")
                        channel_data[channel] = np.array([])
                except Exception as e:
                    print(f"Error acquiring data from {channel}: {e}")
                    channel_data[channel] = np.array([])

            if data_acquired:
                # Compute features for CH1 and CH2
                features = {}
                for channel in ['CH1', 'CH2']:
                    if channel_data[channel].size > 0:
                        channel_features = compute_statistical_features(channel_data[channel], decimation_factor)
                        # Prefix feature names with channel identifier
                        for key, value in channel_features.items():
                            features[f'{channel.lower()}_{key}'] = value
                    else:
                        # Assign NaN to all features if no data
                        for key in [
                            'entropy', 'skewness', 'interquartile_range', 'kurtosis',
                            'percentile_75', 'range', 'maximum', 'median',
                            'percentile_90', 'mean_absolute_deviation'
                        ]:
                            features[f'{channel.lower()}_{key}'] = np.nan

                # Compute moving average of current and previous features
                if previous_features is not None:
                    averaged_features = {}
                    for key in features:
                        # Handle NaN values
                        if np.isnan(features[key]) or np.isnan(previous_features[key]):
                            averaged_features[key] = np.nan
                        else:
                            averaged_features[key] = (features[key] + previous_features[key]) / 2
                else:
                    # For the first iteration, use the current features as they are
                    averaged_features = features.copy()

                # Get current timestamp
                timestamp = datetime.datetime.now().isoformat()

                # Perform prediction using averaged features
                try:
                    predicted_labels, label_probabilities = predict_labels(averaged_features, model, scaler, mlb)
                except ValueError as ve:
                    print(f"Prediction Error: {ve}")
                    label_probabilities = {}
                    predicted_labels = []
                except Exception as e:
                    print(f"An error occurred during prediction: {e}")
                    label_probabilities = {}
                    predicted_labels = []

                # Append averaged features and label probabilities to CSV
                append_payload_to_csv('hello.csv', timestamp, averaged_features, label_probabilities)

                # Send averaged features and label probabilities to AWS
                send_features_to_aws(api_endpoint, api_key, timestamp, averaged_features, label_probabilities)

                # Update previous features for the next iteration
                previous_features = features.copy()

            # Delay before next acquisition cycle
            time.sleep(session_delay)
    except KeyboardInterrupt:
        print("\nData acquisition stopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("Data acquisition terminated.")

# ---------------------------
# Entry Point
# ---------------------------

if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)

    data_acquisition(config)


2024-11-20 01:30:00.875140: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-11-20 01:30:00.875160: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-11-20 01:30:00.875164: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-11-20 01:30:00.875333: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-20 01:30:00.875345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Connected to Red Pitaya at 192.168.8.214
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Payload appended to hello.csv successfully.
{'body': '{"timestamp": "2024-11-20T01:30:02.622514", "features": {"ch1_entropy": 6.056369994855244, "ch1_skewness": 1.7035017490312236, "ch1_interquartile_range": 10.248282654252478, "ch1_kurtosis": 3.7697507011641056, "ch1_percentile_75": -92.19407123652717, "ch1_range": 54.181832161857535, "ch1_maximum": -55.141051995063386, "ch1_median": -98.98544676481585, "ch1_percentile_90": -85.07024528399829, "ch1_mean_absolute_deviation": 6.9655677202688855, "ch2_entropy": 6.059428685990204, "ch2_skewness": 2.066797705911489, "ch2_interquartile_range": 7.279434587934674, "ch2_kurtosis": 6.529416249227735, "ch2_percentile_75": -123.3426948047414, "ch2_range": 57.913131033357445, "ch2_maximum": -83.10900702897078, "ch2_median

2024-11-20 01:30:02.682478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


{"statusCode": 200, "body": "\"Data successfully stored in DynamoDB\""}
Starting data acquisition cycle.
Red Pitaya configured successfully.
Acquisition triggered successfully.
Data acquired from CH1
Data acquired from CH2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Payload appended to hello.csv successfully.
{'body': '{"timestamp": "2024-11-20T01:30:05.689222", "features": {"ch1_entropy": 6.0563634820179, "ch1_skewness": 1.6932555957329796, "ch1_interquartile_range": 10.00177621384411, "ch1_kurtosis": 3.784098133644288, "ch1_percentile_75": -92.27889250691968, "ch1_range": 54.52231232033715, "ch1_maximum": -55.16269017779229, "ch1_median": -99.02860135389284, "ch1_percentile_90": -85.40474593286629, "ch1_mean_absolute_deviation": 6.955566463097771, "ch2_entropy": 6.059437742610802, "ch2_skewness": 2.0322098156524144, "ch2_interquartile_range": 7.099250434395465, "ch2_kurtosis": 6.509189739681963, "ch2_percentile_75": -123.44126074011498, "ch2_range": 57.46836159432709, "ch2_maximum": -83.08

In [ ]:
# AWS connection test script with dummy data 
# expected output, dump dummy data into s3 test-json-storage bucket

import os
import json
import requests
import datetime

def load_config(config_path='config.json'):
    """
    Load configuration parameters from a JSON file.
    """
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    # Validate required keys
    required_keys = ["aws_api_endpoint", "aws_api_key"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config

def generate_dummy_payload():
    """
    Generate a dummy payload that matches the Lambda test event.
    """

    payload = {
        "body": json.dumps({
            "timestamp": "2024-11-15T19:50:08.211911",
            "features": {
                "ch1_entropy": 0.543533
            }
        }),
    }
    return payload

def send_dummy_data(api_endpoint, api_key):
    try:
        # Generate dummy payload matching the test event
        payload = generate_dummy_payload()
        
        # Prepare the request headers
        headers = {
            'Content-Type': 'application/json',
            'x-api-key': api_key  # Include API Key if required
        }
        
        # Print the request details for debugging
        print("Request Headers:")
        print(json.dumps(headers, indent=2))
        print("\nRequest Body:")
        print(json.dumps(payload, indent=2))
        
        # Send POST request to the API endpoint
        response = requests.post(api_endpoint, data=json.dumps(payload), headers=headers)
        
        # Print the response
        print("\nResponse Status Code:", response.status_code)
        if response.headers.get('Content-Type') == 'application/json':
            print("Response Body:", response.json())
        else:
            print("Response Body:", response.text)
    except Exception as e:
        print(f"Error sending dummy data to AWS: {e}")

if __name__ == "__main__":
    try:
        config = load_config('config.json')
        api_endpoint = config['aws_api_endpoint']
        api_key = config['aws_api_key']
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        exit(1)
    
    send_dummy_data(api_endpoint, api_key)


Request Headers:
{
  "Content-Type": "application/json",
  "x-api-key": "xrOcf2c9x778pb76Fojpv7cAUizmiQyl8aIm3Gcd"
}

Request Body:
{
  "body": "{\"timestamp\": \"2024-11-15T19:50:08.211911\", \"features\": {\"ch1_entropy\": 0.543533}}"
}

Response Status Code: 200
Response Body: {'statusCode': 200, 'body': '{"message": "File saved", "file_name": "211df1f8-3fe3-4d31-89ea-95bd034d24df.json"}'}
